+ #### import lib

In [139]:
import pandas as pd
import numpy as np
import mysql.connector
import re
import ast

### read csv file contain dataFrame

In [140]:
top_250 = pd.read_csv('top_250.csv')

1. # Prepairing Data

#### cleaning DataFrame

In [141]:
top_250['parental_guide'].replace(['blank' , 'Not Rated' , 'null' , 'NaN' , 'None'] , 'Unrated' , inplace=True)
top_250['parental_guide'].fillna('Unrated' , inplace=True)

In [142]:
top_250['gross_us_canada']=top_250['gross_us_canada'].str.replace('[,|$]', '' ,regex=True)
top_250['gross_us_canada'].fillna('0' , inplace=True)

In [143]:
def convert_runtime(runtime):
    match = re.match(r'(\d+)h?\s*(\d*)m?', runtime)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2)) if match.group(2) else 0
        return int(hours * 60 + minutes) 

In [144]:
top_250['runtime'] = top_250['runtime'].apply(convert_runtime)

### combine all role of person

In [145]:
all_persons = {}
for column in ['stars', 'writers', 'directors']:
    top_250[column] = top_250[column].apply(ast.literal_eval)
    for row in top_250[column]:
        for key, value in row.items():
            if key not in all_persons:
                all_persons[key] = value

In [146]:
unique_persons = pd.DataFrame(list(all_persons.items()), columns=['Person_ID', 'Name'])

In [147]:
unique_persons.drop_duplicates(inplace=True)

#### cast dataFrame

In [148]:
cast_data = []

for index, row in top_250.iterrows():
    film_id = row["film_id"]  
    stars_dict = row["stars"]
    for star_id in stars_dict.keys():
        cast_data.append({"film_id": film_id, "person_id": star_id})

        
cast_df = pd.DataFrame(cast_data)


#### Crew DataFrame

In [149]:
crew_data = []

for index , row in top_250.iterrows():
    film_id = row["film_id"]
    directors_dict = row["directors"]
    writers_dict = row["writers"]
    for director_id, director_name in directors_dict.items():
        crew_data.append({"film_id": film_id, "person_id": director_id, "role": "Director"})
    for writer_id, writer_name in writers_dict.items():
        crew_data.append({"film_id": film_id, "person_id": writer_id, "role": "Writer"})


crew_df = pd.DataFrame(crew_data)
    

### cleaning genre list from ' and []

+ genre DataFrame

In [172]:
genre_data = []

def clean_genre(genre):
    genre = genre.replace('[', '').replace(']', '').replace("'", "").strip()
    return genre
for index, row in top_250.iterrows():
    film_id = row["film_id"]
    genres = row["genre"]
    genre_list = [clean_genre(g) for g in genres.split(',')]
    for genre in genre_list:
        genre_data.append({"film_id": film_id, "genre": genre})
genre_df = pd.DataFrame(genre_data)

2. # Connect To Database

In [151]:
database = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Sh@0831@Sh'
)

In [152]:
mycursor = database.cursor()

In [153]:

mycursor.execute('create database IMDB_TOP_250')


+ ### use database

In [154]:
mycursor.execute('USE IMDB_TOP_250')

+ ### Now Let Create Tables

In [155]:
mycursor.execute('create table  movie (id VARCHAR(8) PRIMARY KEY , \
                 Title VARCHAR(128) NOT NULL,\
                 Year INT,\
                 Runtime INT,\
                 Parental_guide VARCHAR(8),\
                 Gross_us_canada INT)')


In [156]:
mycursor.execute('create table person (id varchar(8) PRIMARY KEY ,\
                 Name varchar(32))')

In [157]:
mycursor.execute('CREATE TABLE cast (id INT AUTO_INCREMENT PRIMARY KEY,\
    Movie_id VARCHAR(8),\
    Person_id VARCHAR(8),\
    FOREIGN KEY (Movie_id) REFERENCES movie(id),\
    FOREIGN KEY (Person_id) REFERENCES person(id)\
) AUTO_INCREMENT = 1')

In [158]:
mycursor.execute('CREATE TABLE crew (id INT AUTO_INCREMENT PRIMARY KEY,\
                 Movie_id VARCHAR(8),\
                 Person_id VARCHAR(8),\
                 role VARCHAR(8),\
                 FOREIGN KEY (Movie_id) REFERENCES movie(id),\
                 FOREIGN KEY (Person_id) REFERENCES person(id)\
                 ) AUTO_INCREMENT = 1')

In [159]:
mycursor.execute('CREATE TABLE genre_movie (id INT AUTO_INCREMENT PRIMARY KEY, \
                  Movie_id VARCHAR(8),\
                  Genre VARCHAR(16),\
                 FOREIGN KEY (Movie_id) REFERENCES movie(id)\
                 ) AUTO_INCREMENT = 1')

#### add data from dataframe into database

In [160]:
top_250.isna().sum()

title              0
runtime            0
parental_guide     0
year               0
genre              0
directors          0
stars              0
writers            0
gross_us_canada    0
film_id            0
dtype: int64

+ ### Now split values we need from DataFrame and Insert Into DataBase 

+ Movie Table

In [161]:
movie_id = top_250['film_id'].to_list()
titles = top_250['title'].tolist()
years = top_250['year'].tolist()
runtimes = top_250['runtime'].tolist()
parental_guides = top_250['parental_guide'].tolist()
gross_us_canadas = top_250['gross_us_canada'].tolist()

In [162]:
values = list(zip(movie_id,titles, years, runtimes, parental_guides, gross_us_canadas))

In [163]:
sql = "INSERT INTO movie (id,Title, Year, Runtime, Parental_guide, Gross_us_canada) VALUES (%s,%s, %s, %s, %s, %s)"
mycursor.executemany(sql, values)
database.commit()

+ Person Table

In [164]:
values = list(zip(person_id,person_name))
person_id = unique_persons['Person_ID'].tolist()
person_name = unique_persons['Name'].tolist()

In [165]:
sql = "INSERT INTO person (id,Name) VALUES (%s,%s)"
mycursor.executemany(sql, values)
database.commit()

+ Cast Table

In [166]:
cast_movie_id = cast_df['film_id']
cast_id_by_movie = cast_df['person_id']

In [167]:
values = list(zip(cast_movie_id,cast_id_by_movie))
sql = "INSERT INTO cast (Movie_id,Person_id) VALUES (%s,%s)"
mycursor.executemany(sql, values)
database.commit()

+ Crew Table

In [168]:
crew_person_id = crew_df['person_id']
crew_film_id = crew_df['film_id']
crew_role = crew_df['role']


In [169]:
values = list(zip(crew_film_id,crew_person_id,crew_role))
sql = "INSERT INTO crew (Movie_id,Person_id , Role) VALUES (%s,%s,%s)"
mycursor.executemany(sql, values)
database.commit()

+ Genre Table

In [170]:
genre_film_id = genre_df['film_id']
genre_name = genre_df['genre']

In [171]:
values = list(zip(genre_film_id,genre_name))
sql = "INSERT INTO genre_movie (Movie_id,Genre) VALUES (%s,%s)"
mycursor.executemany(sql, values)
database.commit()